In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Dec 23 20:13:27 2024

@author: 14421
"""
# 导入必要的库
import pandas as pd  # 用于数据处理和分析
import matplotlib.pyplot as plt  # 用于绘制图表
import seaborn as sns  # 用于绘制更美观的图表
from sklearn.linear_model import LogisticRegression  # 用于逻辑回归模型
from sklearn.metrics import accuracy_score, classification_report  # 用于评估模型性能
from sklearn.model_selection import train_test_split  # 用于数据集划分
import logging  # 用于日志记录
import os  # 用于操作系统相关的操作

# 配置日志记录器，设置日志级别和格式
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def load_and_preprocess_data(file_path):
    """加载并预处理数据。"""
    try:
        # 从CSV文件中读取数据
        data = pd.read_csv(file_path)
        
        # 要删除的列标签
        labels_to_drop = ['RowNumber', 'CustomerId', 'Surname']
        
        # 删除不需要的列
        data = data.drop(labels=labels_to_drop, axis=1)
        
        # 将分类变量转换为独热编码
        data = pd.get_dummies(data, columns=['Geography', 'Gender'])
        
        # 根据年龄和信用分数筛选数据
        data = data[(data['Age'] <= 60) & (data['CreditScore'] >= 400)]
        
        # 记录日志：数据加载和预处理成功
        logging.info("Data loaded and preprocessed successfully.")
        
        # 返回预处理后的数据
        return data
    except Exception as e:
        # 记录日志：数据加载出错
        logging.error(f"Error loading data: {e}")
        # 抛出异常
        raise

def train_logistic_regression(X, y):
    """训练逻辑回归模型。"""
    # 初始化逻辑回归模型，设置最大迭代次数和随机种子
    model = LogisticRegression(max_iter=1000, random_state=42)
    
    # 使用数据训练模型
    model.fit(X, y)
    
    # 记录日志：逻辑回归模型训练完成
    logging.info("Logistic regression model trained.")
    
    # 返回训练好的模型
    return model

def evaluate_model(model, X, y):
    """评估模型性能并返回准确率。"""
    # 使用模型进行预测
    y_pred = model.predict(X)
    
    # 计算准确率
    accuracy = accuracy_score(y, y_pred)
    
    # 生成分类报告
    report = classification_report(y, y_pred, target_names=['Not Exited', 'Exited'])
    
    # 记录日志：模型评估完成，显示准确率和分类报告
    logging.info(f"Model evaluation completed. Accuracy: {accuracy:.4f}")
    logging.info(f"Classification Report:\n{report}")
    
    # 返回准确率
    return accuracy

def plot_histogram(y_pred_proba, save_path):
    """
    绘制预测概率的直方图并保存。
    """
    # 设置绘图大小
    plt.figure(figsize=(10, 6))
    
    # 绘制直方图
    plt.hist(y_pred_proba, bins=20, edgecolor='k', alpha=0.7)
    
    # 设置x轴和y轴标签
    plt.xlabel('Predicted Churn Probability')
    plt.ylabel('Frequency')
    
    # 设置标题
    plt.title('Logistic Regression Predicted Probability Histogram')
    
    # 显示网格
    plt.grid(axis='y')
    
    # 调整布局
    plt.tight_layout()
    
    # 保存图表
    plt.savefig(save_path)
    
    # 关闭图表
    plt.close()
    
    # 记录日志：直方图已保存
    logging.info(f"Histogram saved to {save_path}.")

def plot_feature_importance(model, X, save_path):
    """
    绘制特征重要性图表并保存。
    """
    # 获取模型的系数
    coefficients = model.coef_[0]
    
    # 创建特征重要性数据框
    feature_importance = pd.DataFrame({'Feature': X.columns, 'Importance': abs(coefficients)})
    
    # 按重要性降序排列
    feature_importance = feature_importance.sort_values(by='Importance', ascending=False)
    
    # 设置绘图大小
    plt.figure(figsize=(10, 6))
    
    # 使用Seaborn绘制条形图
    sns.barplot(x='Importance', y='Feature', data=feature_importance, palette='viridis')
    
    # 设置x轴和y轴标签
    plt.xlabel('Feature Importance (Absolute Coefficient)')
    plt.ylabel('Feature')
    
    # 设置标题
    plt.title('Logistic Regression Feature Importance')
    
    # 显示网格
    plt.grid(axis='x')
    # 调整图像布局，确保标签和标题不会超出图像边界
    plt.tight_layout()
    # 保存图像到指定的路径
    plt.savefig(save_path)
    plt.close()
    logging.info(f"Feature importance plot saved to {save_path}.")
def main():
    # Paths
    file_path = r"C:\Users\14421\Desktop\银行客户流失\代码及数据\Churn_Modelling.csv"
    output_dir = "D:\\数据挖掘11.5"

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
# 加载并预处理数据
    data = load_and_preprocess_data(file_path)
    # 分离特征和目标变量
    X = data.drop('Exited', axis=1)
    y = data['Exited']

    # 划分数据集为训练集和测试集
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42)

    # 训练逻辑回归模型并在测试集上评估模型
    logreg_model = train_logistic_regression(X_train, y_train)
    # 获取测试集的预测概率
    y_pred_proba = logreg_model.predict_proba(X_test)[:, 1]  # 注意修正索引以获取正确的概率
    # 计算模型准确度
    accuracy = evaluate_model(logreg_model, X_test, y_test)

    # 绘制并保存预测概率的直方图
    plot_histogram(y_pred_proba, os.path.join(output_dir, "logreg_histogram.png"))
    # 绘制并保存特征重要性图表
    plot_feature_importance(logreg_model, X, os.path.join(output_dir, "logreg_feature_importance.png"))

if __name__ == "__main__":
    main()
  